In [1]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
from feature_extraction import get_all_features
import pickle
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
with open('lmart.pkl', 'rb') as f:
    lmart = pickle.load(f)

In [3]:
RUN_NAME = 'ows_ltr'
RUN_DIR = Path('../runs/')

In [4]:
def create_run(dataset_name: str, run_suffix: str):
    dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset_name}')

    extract_features = get_all_features(tira, dataset)

    bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

    pipeline = (bm25 % 1000) >> extract_features >> pt.ltr.apply_learned_model(lmart, form="ltr")

    run = pipeline(dataset.get_topics(variant='query'))

    run_file = RUN_DIR / RUN_NAME / f'{RUN_NAME}.{run_suffix}'
    run_file.parent.mkdir(exist_ok=True, parents=True)

    pt.io.write_results(run, run_file, format='trec', run_name=RUN_NAME)

In [5]:
DATASETS = {
    'longeval-train-20230513-training': 'train_2023', 
    'longeval-heldout-20230513-training': 'WT',
    'longeval-short-july-20230513-training': 'ST',
    'longeval-long-september-20230513-training': 'LT',
    'longeval-2023-01-20240423-training': 'train_2024',
    'longeval-2023-06-20240418-training': 'lag6',
    'longeval-2023-08-20240418-training': 'lag8',
}

In [7]:
for dataset_name, suffix in DATASETS.items():
    print(f'Running {dataset_name}...')
    create_run(dataset_name, suffix)

Running longeval-train-20230513-training...
Running longeval-heldout-20230513-training...
Running longeval-short-july-20230513-training...
Running longeval-long-september-20230513-training...
Running longeval-2023-01-20240423-training...
Running longeval-2023-06-20240418-training...
Running longeval-2023-08-20240418-training...
